In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import json
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
all_files = glob.glob("/content/drive/MyDrive/kaggle_dataset/*.csv") # all .csv files
all_files.sort() # imposing a given order
print(all_files)

['/content/drive/MyDrive/kaggle_dataset/Albuquerque_Final_2022-06-18.csv', '/content/drive/MyDrive/kaggle_dataset/Anaheim_Final_2022-06-18.csv', '/content/drive/MyDrive/kaggle_dataset/Arlington_Final_2022-06-18.csv', '/content/drive/MyDrive/kaggle_dataset/Atlanta_Final_2022-06-18.csv', '/content/drive/MyDrive/kaggle_dataset/AuroraCO_Final_2022-06-18.csv', '/content/drive/MyDrive/kaggle_dataset/Austin_Final_2022-06-18.csv', '/content/drive/MyDrive/kaggle_dataset/Baltimore_Final_2022-06-18.csv', '/content/drive/MyDrive/kaggle_dataset/Boston_Final_2022-06-18.csv', '/content/drive/MyDrive/kaggle_dataset/Buffalo_Final_2022-06-18.csv', '/content/drive/MyDrive/kaggle_dataset/CapeCoral_Final_2022-06-18.csv', '/content/drive/MyDrive/kaggle_dataset/ColoradoSprings_Final_2022-06-18.csv', '/content/drive/MyDrive/kaggle_dataset/Columbus_Final_2022-06-18.csv', '/content/drive/MyDrive/kaggle_dataset/Column_Headers_Dryad.csv', '/content/drive/MyDrive/kaggle_dataset/Dallas_Final_2022-06-18.csv', '/cont

In [ ]:
# Create a list to store DataFrames
list_of_df = []

# Load and preprocess the CSV files
for filename in all_files:
    df = pd.read_csv(filename, low_memory=False)

    # Check if 'city' column exists in the DataFrame
    if 'city' in df.columns:
        # Extract the city name from the filename
        city_name = filename.split('_')[0].split('/')[-1]
        # Fill missing 'city' values with the extracted city name
        df['city'].fillna(city_name, inplace=True)

    list_of_df.append(df)

In [ ]:
# Concatenate all DataFrames into a single DataFrame
tree_dataset = pd.concat(list_of_df, axis=0, ignore_index=True)

# Filter for rows with both 'city' and 'scientific_name'
filtered_data = tree_dataset[['city', 'scientific_name', 'height_M']]

# Count the occurrences of 'scientific_name' based on 'city'
scientific_name_counts = filtered_data.groupby(['city', 'scientific_name']).size().reset_index(name='count')

# Calculate the average height for each scientific name
average_heights = filtered_data.groupby(['scientific_name'])['height_M'].mean().reset_index(name='average_height')

# Sort the counts by city and count in descending order
sorted_counts = scientific_name_counts.sort_values(by=['city', 'count'], ascending=[True, False])

In [ ]:
# Create a dictionary to store the top 7 counts for each city and the average height for each scientific name
top_counts = {}
for city in sorted_counts['city'].unique():
    city_data = sorted_counts[sorted_counts['city'] == city]
    top_30 = city_data.head(30).to_dict(orient='records')
    top_counts[city] = top_30

# Create a dictionary to store the average height for each scientific name
average_height_dict = average_heights.set_index('scientific_name')['average_height'].to_dict()

In [ ]:
# Combine the two dictionaries and filter out entries with NaN average_height
final_result = {}
for city, top_scientific_names in top_counts.items():
    final_result[city] = []
    for name_info in top_scientific_names:
        scientific_name = name_info['scientific_name']
        count = name_info['count']
        if scientific_name in average_height_dict:
            average_height = average_height_dict[scientific_name]
            if not pd.isna(average_height):
                final_result[city].append({
                    'scientific_name': scientific_name,
                    'count': count,
                    'average_height': average_height
                })

# Save the final result as a JSON file
with open('top_city_scientific_names_with_average_height.json', 'w') as json_file:
    json.dump(final_result, json_file, indent=4)

In [ ]:
import pandas as pd

# Combine the data into a DataFrame
final_data = []
for city, top_scientific_names in final_result.items():
    for item in top_scientific_names:
        final_data.append({
            'City': city,
            'Scientific_Name': item['scientific_name'],
            'Count': item['count'],
            'Average_Height': item['average_height']
        })

df = pd.DataFrame(final_data)

# Save the DataFrame as a CSV file
df.to_csv('top_city_scientific_names_with_average_height.csv', index=False)